In [2]:
import Auto_paint_self
more_class_true,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one','E:/unet_classify_testy_more_classss.nc','testy','yes','time','2017-10-24','2021-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
more_class_pred,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one','E:/unet_classify_predicty_more_classss.nc','testy','yes','time','2017-10-24','2021-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
one_class_true,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one','E:/unet_classify_testy.nc','testy','yes','time','2017-10-24','2021-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
one_class_pred,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one','E:/unet_classify_predicty.nc','testy','yes','time','2017-10-24','2021-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')

F:\anaconda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [10]:
CN05_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\CN05.1_Pre_1961_2021_daily_025x025.nc','pre','yes','time','2001-01-01','2021-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
CN05_PRE=CN05_PRE[:,::-1,:]

C:\Users\TBYC\AppData\Roaming\Python\Python311\site-packages\xarray\coding\times.py:206: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  ref_date = _ensure_padded_year(ref_date)


In [3]:
import numpy as np
from sklearn.metrics import (
    matthews_corrcoef,
    balanced_accuracy_score,
    cohen_kappa_score,
    accuracy_score,
)

def _flatten_pair(y_true, y_pred, sample_weight=None, ignore_values=None):
    """
    将三维(y_true, y_pred)展平为一维，同时应用掩码：
    - 忽略 NaN
    - 忽略在 ignore_values 中的值（可选）
    - sample_weight 若提供，需可广播或同形状
    返回: y_true_1d, y_pred_1d, w_1d
    """
    if y_true.shape != y_pred.shape:
        raise ValueError(f"Shape mismatch: y_true {y_true.shape} vs y_pred {y_pred.shape}")

    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    # 初始有效掩码：非 NaN
    mask = np.ones(y_true.shape, dtype=bool)
    if np.issubdtype(y_true.dtype, np.floating):
        mask &= ~np.isnan(y_true)
    if np.issubdtype(y_pred.dtype, np.floating):
        mask &= ~np.isnan(y_pred)

    # 忽略指定的无效值
    if ignore_values is not None:
        mask &= ~np.isin(y_true, ignore_values)
        mask &= ~np.isin(y_pred, ignore_values)

    y_true_1d = y_true[mask].ravel()
    y_pred_1d = y_pred[mask].ravel()

    if y_true_1d.size == 0:
        raise ValueError("No valid samples after masking.")

    w_1d = None
    if sample_weight is not None:
        w = np.broadcast_to(np.asarray(sample_weight), y_true.shape)[mask].ravel()
        w_1d = w.astype(float)

    return y_true_1d, y_pred_1d, w_1d


def mcc_score(y_true, y_pred, sample_weight=None, ignore_values=None):
    y_t, y_p, w = _flatten_pair(y_true, y_pred, sample_weight, ignore_values)
    return matthews_corrcoef(y_t, y_p, sample_weight=w)


def kappa_score(y_true, y_pred, sample_weight=None, ignore_values=None, weights=None):
    """
    weights=None: 名义型类别的普通kappa
    weights="linear"或"quadratic": 若类别有序，可用加权kappa
    """
    y_t, y_p, w = _flatten_pair(y_true, y_pred, sample_weight, ignore_values)
    return cohen_kappa_score(y_t, y_p, weights=weights, sample_weight=w)


def nba_score(y_true, y_pred, K=None, sample_weight=None, ignore_values=None):
    """
    NBA = (BA - 1/K) / (1 - 1/K)，其中 BA 是宏平均召回（balanced accuracy）。
    若未给 K，则用 y_true/ y_pred 的标签并集个数推断。
    """
    y_t, y_p, w = _flatten_pair(y_true, y_pred, sample_weight, ignore_values)
    ba = balanced_accuracy_score(y_t, y_p, sample_weight=w)

    if K is None:
        K = np.unique(np.concatenate([np.unique(y_t), np.unique(y_p)])).size
    if K < 2:
        raise ValueError(f"K must be >=2, got {K}")

    baseline = 1.0 / K
    return (ba - baseline) / (1.0 - baseline)


def _class_prevalence(y, sample_weight=None):
    """
    返回类别标签数组 labs 以及对应的加权频率 p_i（按 y 的真实分布）。
    """
    labs, inv = np.unique(y, return_inverse=True)
    if sample_weight is None:
        counts = np.bincount(inv)
    else:
        counts = np.bincount(inv, weights=sample_weight)
    p = counts / counts.sum()
    return labs, p


def acc_skill_score(y_true, y_pred, baseline="prevalence", K=None,
                    sample_weight=None, ignore_values=None):
    """
    Accuracy Skill Score:
      AccSS = (Acc - Acc_baseline) / (1 - Acc_baseline)
    baseline 选项：
      - "prevalence": 按真实先验频率随机猜测，基线 = sum_i p_i^2（推荐）
      - "majority": 永远猜多数类
      - "uniform": 均匀随机猜测，基线 = 1/K
    返回可能为负（模型比基线还差）。
    """
    y_t, y_p, w = _flatten_pair(y_true, y_pred, sample_weight, ignore_values)
    acc = accuracy_score(y_t, y_p, sample_weight=w)

    if baseline == "prevalence":
        _, p = _class_prevalence(y_t, w)
        acc_base = np.sum(p ** 2)
    elif baseline == "majority":
        _, p = _class_prevalence(y_t, w)
        acc_base = p.max()
    elif baseline == "uniform":
        if K is None:
            K = np.unique(np.concatenate([np.unique(y_t), np.unique(y_p)])).size
        acc_base = 1.0 / K
    else:
        raise ValueError("baseline must be one of {'prevalence','majority','uniform'}")

    if acc_base >= 1.0:
        # 极端情况（全同一类且K=1），无法定义技能分数
        return np.nan

    return (acc - acc_base) / (1.0 - acc_base)


def metrics_over_time(y_true, y_pred, K=None, sample_weight_3d=None,
                      ignore_values=None, kappa_weights=None):
    """
    沿 time 维逐时计算指标。返回一个 dict，键是指标名，值是 shape=(time,) 的数组。
    y_true/y_pred: shape=(time, lat, lon)
    sample_weight_3d: 可选，shape 可广播到 (time, lat, lon)
    """
    if y_true.shape != y_pred.shape or y_true.ndim != 3:
        raise ValueError("y_true and y_pred must both be 3D with same shape (time, lat, lon).")

    T = y_true.shape[0]
    out = {
        "MCC": np.full(T, np.nan, float),
        "Kappa": np.full(T, np.nan, float),
        "NBA": np.full(T, np.nan, float),
        "AccSS_prev": np.full(T, np.nan, float),
    }
    for t in range(T):
        sw_t = None
        if sample_weight_3d is not None:
            sw_t = np.broadcast_to(sample_weight_3d, y_true.shape)[t]
        try:
            out["MCC"][t] = mcc_score(y_true[t], y_pred[t], sample_weight=sw_t, ignore_values=ignore_values)
            out["Kappa"][t] = kappa_score(y_true[t], y_pred[t], sample_weight=sw_t,
                                          ignore_values=ignore_values, weights=kappa_weights)
            out["NBA"][t] = nba_score(y_true[t], y_pred[t], K=K, sample_weight=sw_t, ignore_values=ignore_values)
            out["AccSS_prev"][t] = acc_skill_score(y_true[t], y_pred[t], baseline="prevalence",
                                                   K=K, sample_weight=sw_t, ignore_values=ignore_values)
        except ValueError:
            # 某一时刻全部被掩蔽或无有效样本
            pass
    return out

In [13]:
print(CN05_PRE.shape,more_class_true.shape,more_class_pred.shape,one_class_true.shape,one_class_pred.shape)

(7670, 143, 247) (1530, 143, 247) (1530, 143, 247) (1530, 143, 247) (1530, 143, 247)


In [14]:
more_class_true_no_nan=np.zeros((more_class_true.shape[0],more_class_true.shape[1],more_class_true.shape[2]))
more_class_pred_no_nan=np.zeros((more_class_pred.shape[0],more_class_pred.shape[1],more_class_pred.shape[2]))
one_class_true_no_nan=np.zeros((one_class_true.shape[0],one_class_true.shape[1],one_class_true.shape[2]))
one_class_pred_no_nan=np.zeros((one_class_pred.shape[0],one_class_pred.shape[1],one_class_pred.shape[2]))
for i in range(CN05_PRE.shape[1]):
    for j in range(CN05_PRE.shape[2]):
        if np.isnan(CN05_PRE[:,i,j]).any():
            more_class_true_no_nan[:,i,j]=np.nan
            more_class_pred_no_nan[:,i,j]=np.nan
            one_class_true_no_nan[:,i,j]=np.nan
            one_class_pred_no_nan[:,i,j]=np.nan

In [15]:
# 多分类
mcc_more   = mcc_score(more_class_true, more_class_pred)
kappa_more = kappa_score(more_class_true, more_class_pred)  
nba_more   = nba_score(more_class_true, more_class_pred, K=6)

# 二分类
mcc_one   = mcc_score(one_class_true, one_class_pred)
kappa_one = kappa_score(one_class_true, one_class_pred)
nba_one   = nba_score(one_class_true, one_class_pred, K=2)

In [16]:
print(mcc_more,kappa_more,nba_more)
print(mcc_one,kappa_one,nba_one)

0.012273107084098697 0.000555286228196672 0.00019622641071899148
0.722256970236556 0.7217996165828908 0.7073862976276846


In [6]:
def Auto_radar_plot(v, labels, selfcolor=None, legend=None, labelsize=20, linewidth=2, vmin=None, vmax=None, space=None, title=' ', ifsave='no', savename=None):
    import numpy as np
    import matplotlib.pyplot as plt

    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False

    v_array = np.array(v)
    labels_array = np.array(labels)
    num_vars = len(labels_array)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False)
    angles_closed = np.concatenate([angles, [angles[0]]])

    # 自动颜色: 支持多组或单组
    if selfcolor is None:
        colorlist = ['skyblue', 'salmon', 'limegreen', 'orange', 'violet', 'gold', 'deepskyblue']
    elif isinstance(selfcolor, (list, tuple, np.ndarray)):
        colorlist = list(selfcolor)
    else:
        colorlist = [selfcolor]
    
    # 保证 v_array 是二维
    if v_array.ndim == 1:
        v_array = v_array[np.newaxis, :]
    n_group = v_array.shape[0]

    fig, ax = plt.subplots(subplot_kw={'polar': True}, figsize=(8, 8), facecolor='white')
    ax.spines['polar'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])

    for i in range(n_group):
        values = v_array[i]
        values_closed = np.concatenate([values, [values[0]]])
        color = colorlist[i % len(colorlist)]
        label = None
        if legend is not None and i < len(legend):
            label = legend[i]
        ax.plot(angles_closed, values_closed, 'o-', linewidth=linewidth, color=color, label=label)
        ax.fill(angles_closed, values_closed, color=color, alpha=0.25)
    
    ax.set_thetagrids(np.degrees(angles), labels_array, fontsize=labelsize)
    ax.xaxis.set_tick_params(pad=15+(labelsize-20)*1)
    all_values = v_array.flatten()
    if vmax is None and vmin is None:
        ax.set_ylim(np.nanmin(all_values), np.nanmax(all_values))
    else:
        ax.set_ylim(vmin, vmax)
    ax.grid(True)
    if vmax is None and vmin is None and space is None:
        ax.set_rgrids(np.linspace(np.nanmin(all_values), np.nanmax(all_values), 5), fontsize=labelsize-2)
    elif space is None:
        ax.set_rgrids(np.linspace(vmin, vmax, 5), fontsize=labelsize-2)
    elif vmax is None and vmin is None:
        ax.set_rgrids(np.linspace(np.nanmin(all_values), np.nanmax(all_values), space), fontsize=labelsize-2)
    else:
        ax.set_rgrids(np.linspace(vmin, vmax, space), fontsize=labelsize-2)
    ax.set_title(title, y=1.0+labelsize*0.005, fontsize=labelsize)
    
    if legend is not None:
        ax.legend(loc='upper right', bbox_to_anchor=(1.1+(labelsize-20)*0.04, 1.1), fontsize=labelsize-2, frameon=True)
    
    plt.show()
    
    if ifsave == 'yes' and savename is not None:
        fig.savefig(savename, dpi=300)


In [ ]:
Auto_radar_plot([[mcc_more,kappa_more,nba_more],[mcc_one,kappa_one,nba_one]], ['MCC','KAPPA','NBA'], selfcolor=None, legend=['Six Class','Two Class'], labelsize=40, linewidth=2, vmin=None, vmax=None, space=None, title=' ', ifsave='no', savename=None)